In [2]:
x = {184737297734959104 : {'Name': ['adam'],
                        'avaliability': {'2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-08': True,
                                '2023-06-01': True,
                                '2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}  
                            },

     462409072176791564 : {'Name': ['matty'],
                        'avaliability': {'2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-08': True,
                                '2023-06-01': True,
                                '2023-05-25': True},
                        'goal': {'2023-06-08': 2,
                                '2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 1},
                        'motm-vote': {},
                        'motm': {'2023-06-08': 1}                            
                           },

     265573788232581120 : {'Name': ['tommy'],
                        'avaliability': {'2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'no'},
                        'paid': {'2023-06-08': True,
                                '2023-06-01': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}       
                            },   

     454252561063477250 : {'Name': ['noz', 'lawrence'],
                        'avaliability': {'2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-08': True,
                                '2023-06-01': True,
                                        '2023-05-25': True},
                        'goal': {'2023-06-08': 1,
                                 '2023-06-01': 0,
                                '2023-05-25': 1},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0}, 
                        'motm-vote': {},
                        'motm': {}      
                            },  

     462410416048308225 : {'Name': ['tom', 'Tom b'],
                        'avaliability': {'2023-06-01': 'no',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}       
                            },

     462654687225708555 : {'Name': ['tk', 'thomas'],
                        'avaliability': {'2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-08': True,
                                '2023-06-01': True,
                                '2023-05-25': True},
                        'goal': {'2023-06-08': 1,
                                '2023-06-01': 1,
                                '2023-05-25': 1},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}     
                            },

     265576326788808704 : {'Name': ['jack'],
                        'avaliability': {'2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-08': True,
                                '2023-06-01': True,
                                '2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}       
                            }

     }

In [1]:
# autoreload modules
%load_ext autoreload
%autoreload 2

In [3]:
import json

# read data from json
path = '//TRUENAS/Misc_storage/5aside_discord_bot/user_data/team_data.json'

# with open(path, 'r') as f:
#     data = json.load(f)

# save data to json
with open(path, 'w') as f:
    json.dump(x, f, indent=4)




In [40]:
import os
os.chdir(os.path.expanduser("../matches/"))

from league_scraper import get_league_matches

new_data = get_league_matches()

In [15]:
from games import Fixtures

fixtures = Fixtures('', '//TRUENAS/Misc_storage/5aside_discord_bot/')
old_data = fixtures.match_data

In [ ]:
import pandas as pd
from datetime import datetime as dt
from fixture_updates import fixture_data_format

async def check_new_fixture_data(new_data: pd.DataFrame, old_data: pd.DataFrame, 
                                 path: str, bot, channel_id):
    """
    Function to update the fixture data with new data from the latest website scrape.
    There are 3 cases which can occur:

    - case 1: new season started
    - case 2: new results added
        - case 2.1: new team
    - case 3: no update 
    """
    # fix time column (remove seconds)
    new_data = fixture_data_format(new_data)

    # case 1: new season
    if all(new_data['Date'] != old_data['Date']):    
        latest_date = dt.strptime(old_data['Date'].iloc[-1], '%Y-%m-%d')

        #old_season_path = f'//TRUENAS/Misc_storage/5aside_discord_bot/league_data/old_seasons/'
        old_season_path = path+ f'/league_data/old_seasons/'
        old_data.to_csv(old_season_path+f'Season-{latest_date.strftime("%Y-%m-%d")}.csv', index=False)
        
        return new_data
    
    # case 2: check for new results
    elif any(new_data['Pending'] != old_data['Pending']):
        # new results added
        new_results = new_data[new_data['Pending'] != old_data['Pending']]

        # display new results
        latest_results_msg = f'**New results extracted**:\n'
        latest_results_msg += 'The latest fixtures:\n'
        latest_results_msg += new_results[['Time','Home','Home score','Away score','Away']].to_string(index=False)
        
        await bot.send_message(channel_id, latest_results_msg)


        # case 2.1: new team
        old_data_teams = old_data[new_data['Pending'] != old_data['Pending']]
        n_team = set(new_results['Home']).union(set(new_results['Away']))
        o_team = set(old_data_teams['Home']).union(set(old_data_teams['Away']))

        async def new_team(new, old):
            "Helper function - msgs about team change"
            msg = f"\n**New team added**: {new} (previously was {old})\n\n"
            await bot.send_message(channel_id, msg)

        # check for new team in latest fixtures
        for (n_hteam, n_ateam, o_hteam, o_ateam) in zip(new_results['Home'], new_results['Away'], \
                                      old_data_teams['Home'], old_data_teams['Away']):
            
            if (n_hteam != o_hteam) & (o_hteam not in n_team):
                await new_team(n_hteam, o_hteam)

            if (n_ateam != o_ateam) & (o_ateam not in n_team):
                await new_team(n_ateam, o_ateam)



        
    # case 3: no update
    else:
        # msg Admin - tried to pull new results but none found

        print('No new results')


    return 

import copy

await check_new_fixture_data(copy.deepcopy(new_data), old_data, '', '')

: 

In [ ]:
3/13 *195/2

22.5

In [ ]:
old_data.columns

: 

In [26]:
new_data == old_data

,Date,Time,Home,Away,Home score,Away score,Pending,Datetime,Winner
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True


In [1]:
from datetime import datetime as dt
dt.strptime(5, '%Y-%m-%d')

TypeError: strptime() argument 1 must be str, not int